In [ ]:
import cv2
import numpy as np
from keras import Model
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import Callback, EarlyStopping
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import Sequence
import pickle as pkl

from matplotlib import pyplot

# Images Generators

In [ ]:
with open("../102flowers/pkl-1/X_train.pkl", "rb") as f:
  X_train = pkl.load(f)
with open("../102flowers/pkl-1/y_train.pkl", "rb") as f:
  y_train = pkl.load(f)

train_imgs = []
for i, path in enumerate(X_train):
  img = path.split('\\')[-1]
  train_imgs.append(f"../102flowers/102flowers/{img}")

In [ ]:
with open("../102flowers/pkl-1/X_val.pkl", "rb") as f:
  X_val = pkl.load(f)
with open("../102flowers/pkl-1/y_val.pkl", "rb") as f:
  y_val = pkl.load(f)

val_imgs = []
for i, path in enumerate(X_val):
  img = path.split('\\')[-1]
  val_imgs.append(f"../102flowers/102flowers/{img}")

In [ ]:
with open("../102flowers/pkl-1/X_test.pkl", "rb") as f:
  X_test = pkl.load(f)
with open("../102flowers/pkl-1/y_test.pkl", "rb") as f:
  y_test = pkl.load(f)

test_imgs = []
for i, path in enumerate(X_test):
  img = path.split('\\')[-1]
  test_imgs.append(f"../102flowers/102flowers/{img}")

In [ ]:
class FlowersImagesGenerator(Sequence):

  def __init__(self, images_paths, y, num_classes=102, batch_size=32, *args, **kwargs):
    self.images_paths = images_paths
    self.y = y
    self.num_classes = num_classes
    self.batch_size = batch_size

  def __len__(self):
    return (len(self.images_paths) // self.batch_size)

  def __getitem__(self, index):
    batch_images = self.images_paths[self.batch_size*index : self.batch_size*(index+1)]
    y_batch = self.y[self.batch_size*index : self.batch_size*(index+1)]

    X = [cv2.resize(cv2.imread(img_path)/255, (256,256)) for img_path in batch_images]
    y_onehot = np.zeros((y_batch.shape[0], self.num_classes))
    for i, val in enumerate(y_batch):
      y_onehot[i, val - 1] = 1  # y classes are 1 to 102
    return np.array(X), y_onehot

In [ ]:
num_classes = 102

# Inception

In [ ]:
inception = InceptionV3(input_shape=(256,256,3),
                        include_top=False,
                        pooling=max,
                        weights='imagenet',
                        classes=num_classes,
                        classifier_activation='softmax')

In [ ]:
for layer in inception.layers:
  layer.trainable = False

In [ ]:
out = Flatten()(inception.output)
out = Dense(1024, activation='relu', name="fc1")(out)
out = Dropout(0.2)(out)
predictions = Dense(num_classes, activation='softmax', name="predictions")(out)
model = Model(inputs=inception.input, outputs=predictions)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', 'categorical_accuracy'])

In [ ]:
class TestMetricsCallback(Callback):
  def __init__(self):
    self.epoch_test_loss = []
    self.epoch_test_accuracy = []
    self.epoch_test_categorical_accuracy = []

  def on_epoch_end(self, epoch, logs=None):
    test_generator = FlowersImagesGenerator(images_paths=test_imgs, y=y_test)
    loss, accuracy, categorical_accuracy = self.model.evaluate(test_generator)
    self.epoch_test_loss.append(loss)
    self.epoch_test_accuracy.append(accuracy)
    self.epoch_test_categorical_accuracy.append(categorical_accuracy)

In [ ]:
train_generator = FlowersImagesGenerator(images_paths=train_imgs, y=y_train)
val_generator = FlowersImagesGenerator(images_paths=val_imgs, y=y_val)

In [ ]:
early_stop_callback = EarlyStopping(monitor='val_loss', mode='min')
test_metrics_callback = TestMetricsCallback()
history = model.fit(train_generator, validation_data=val_generator, epochs=50,
                    callbacks=[early_stop_callback, test_metrics_callback], verbose=2)

In [ ]:
import json
with open("./history.json", "w+") as f:
  json.dump(history.history, f)

In [ ]:
pyplot.plot(history.history['accuracy'], label='train accuracy')
pyplot.plot(history.history['val_accuracy'], label='val accuracy')
pyplot.plot(test_metrics_callback.epoch_test_accuracy, label='test accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(loc="upper right")
pyplot.show()
pyplot.savefig("accuracy plot")

In [ ]:
pyplot.plot(history.history['categorical_accuracy'], label='train categorical accuracy')
pyplot.plot(history.history['val_categorical_accuracy'], label='val categorical accuracy')
pyplot.plot(test_metrics_callback.epoch_test_categorical_accuracy, label='test categorical accuracy')
pyplot.ylabel('categorical accuracy')
pyplot.xlabel('epoch')
pyplot.legend(loc="upper right")
pyplot.show()
pyplot.savefig("categorical accuracy plot")